## A. Funciones.

In [ ]:
#ii. Evaluación de Ganancia (LGBM).
def lgb_gan_eval(y_pred, data):
    weight = data.get_weight()
    ganancia = np.where(weight == 1.00002, ganancia_acierto, 0) - np.where(weight < 1.00002, costo_estimulo, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)

    return 'gan_eval', np.max(ganancia) , True


In [ ]:
#iii. Probabilidad de Ganancia (LGBM).
def ganancia_prob(y_pred, y_true, prop = 1):
  ganancia = np.where(y_true == 1, ganancia_acierto, 0) - np.where(y_true == 0, costo_estimulo, 0)
  return ganancia[y_pred >= 0.025].sum() / prop

In [ ]:
#iv. Probabilidad de Ganancia (Random Forest).
def ganancia_prob_rf(y_hat, y, prop=1, class_index=1, threshold=0.025):
  @np.vectorize
  def ganancia_row(predicted, actual, threshold=0.025):
    return  (predicted >= threshold) * (ganancia_acierto if actual == "BAJA+2" else -costo_estimulo)

  return ganancia_row(y_hat[:,class_index], y).sum() / prop

In [ ]:
#v. Diferentes funciones y métodos para corregir el efecto de la inflación.
def drift_uva(dataset, campos_monetarios, tb_indices):
    print("inicio drift_UVA()")
    dataset = dataset.merge(tb_indices[['foto_mes', 'UVA']], on='foto_mes', how='left')
    for campo in campos_monetarios:
        dataset[campo] *= dataset['UVA']
    dataset.drop(columns=['UVA'], inplace=True)
    print("fin drift_UVA()")
    return dataset

def drift_deflacion(dataset, campos_monetarios, tb_indices):
    print("inicio drift_deflacion()")
    dataset = dataset.merge(tb_indices[['foto_mes', 'IPC']], on='foto_mes', how='left')
    for campo in campos_monetarios:
        dataset[campo] *= dataset['IPC']
    dataset.drop(columns=['IPC'], inplace=True)
    print("fin drift_deflacion()")
    return dataset

# Función para estandarizar datos
def drift_estandarizar(dataset, campos_drift):
    print("inicio drift_estandarizar()")
    for campo in campos_drift:
        dataset[campo + "_normal"] = dataset.groupby('foto_mes')[campo].transform(lambda x: (x - x.mean()) / x.std())
        dataset.drop(columns=[campo], inplace=True)
    print("fin drift_estandarizar()")
    return dataset

In [ ]:
#vi. Adaptación de la función de ganancia para Random Forest.
def rf_gan_eval(y_true, y_pred_proba, weights):
    # Define los parámetros de ganancia y costo
    ganancia_acierto = 7800  # Ajusta según tus datos
    costo_estimulo = 200  # Ajusta según tus datos
    
    # Calcular la ganancia en base a las predicciones y los pesos
    ganancia = np.where(weights == 1.00002, ganancia_acierto, 0) - np.where(weights < 1.00002, costo_estimulo, 0)
    
    # Ordenar las ganancias de acuerdo a las probabilidades de la clase positiva
    ganancia = ganancia[np.argsort(y_pred_proba)[::-1]]
    
    # Cálculo de ganancia acumulada y obtención de la máxima ganancia
    ganancia_acumulada = np.cumsum(ganancia)
    max_ganancia = np.max(ganancia_acumulada)
    
    return max_ganancia